# Introduction

* router are natively friendly device 
    - receives a packet 
    - look up the routing table 
    - forwards to the appropreate port 
* what if we need a secure forwarding ? 
    - internet uses public IP and networks behind the routers uses private which is not reachable through the internet. malicious user makes the internal network to access a publicy available roughe resource to compromise the source, called _reverse shell_. 
* ACL is more used Internally than Externally 
    - protectig networks from insiders (within your orgnanization) is more challenging 
* ACL is not limited only to security ( _list of permit/deny statement_ )
    - permit/deny an IP address
    - permit/deny a port for a IP address 
* Usage 
    1. Access Control : typicall blocking of traffic 
    2. NAT : which address to be translated 
    3. QoS : Traffic engineering on a specific set of IPs
    4. PBR
    5. Route Filtering 
* ACL rules 
    1. read top-down
    2. order matters, stops at the first match 
    3. Implicit dely at the last 
    4. ACL is applied inbound/outbound, think bounds as your are the router !
    4. systax : `access-list [num] [permit | deny] [ip addr] [wc] `
    5. each enry has a sequence number, increases by 10
* ACL types
    1. Standard (basic) 
        - matches based on sourse address (rarely used for security)
        - lower processor unitlization
    2. Extended (advanced)
        - Matches based on source/destination address, port, protocol 
        - higher processor utilization
    3. Reflexive (Stateful) 
        - Allowes return traffic for internal requests 
    4. Named access-lists 
        - `ip access-list [name] `

![](acl1.png)

## Base config 

```
!r1
conf t
	int f0/0
		ip add 12.0.0.1 255.255.255.0
		no sh
	int e1/0
		ip add 192.168.11.1 255.255.255.0
		no sh
	int e1/1
		ip add 192.168.12.1 255.255.255.0
		no sh
	router eigrp 100
		no auto 
		passive def 
		net 0.0.0.0
		no passive f0/0
end 

!r2
conf t
	int f0/0
		ip add 12.0.0.2 255.255.255.0
		no sh
	int f0/1
		ip add 23.0.0.2 255.255.255.0
		no sh
	int l1
		ip add 192.168.21.1 255.255.255.0
	int l2
		ip add 192.168.22.1 255.255.255.0
	int l3
		ip add 192.168.23.1 255.255.255.0
	router eigrp 100
		no auto 
		passive def 
		net 0.0.0.0
		no passive f0/0
		no passive f0/1
end 

!r3
conf t
	int f0/0
		ip add 23.0.0.3 255.255.255.0
		no sh
	int f0/1
		ip add 192.168.33.1 255.255.255.0
		no sh
	router eigrp 100
		no auto 
		passive def 
		net 0.0.0.0
		no passive f0/0
end 
```

### Ensuring full access
```
web> ping 192.168.12.10 -count 3
84 bytes from 192.168.12.10 icmp_seq=1 ttl=61 time=41.054 ms
84 bytes from 192.168.12.10 icmp_seq=2 ttl=61 time=36.535 ms
84 bytes from 192.168.12.10 icmp_seq=3 ttl=61 time=36.005 ms

web> ping 192.168.11.10 -count 3
84 bytes from 192.168.11.10 icmp_seq=1 ttl=61 time=38.582 ms
84 bytes from 192.168.11.10 icmp_seq=2 ttl=61 time=35.741 ms
84 bytes from 192.168.11.10 icmp_seq=3 ttl=61 time=35.672 ms

PC3> ping 192.168.33.200 -c 3   
84 bytes from 192.168.33.200 icmp_seq=1 ttl=61 time=45.184 ms
84 bytes from 192.168.33.200 icmp_seq=2 ttl=61 time=45.744 ms
84 bytes from 192.168.33.200 icmp_seq=3 ttl=61 time=35.852 ms
```

## Apply Standad access list 

* A generic way to use any access list is __Named Access__ using `ip access-list [type] [name | number]` command
* For standard access list rule-of-thumb apply on farthest interface 
    ```
    int [iface]
        ip access-group [name | number] [in | out]
    ```
* In this examle 
    - Task 1 : R1 is __blocking__ requests from __`192.168.33.1`__ outgoing to __`e1/0`__ 
    - Task 2 : R3 is __blocking__ request from __`192.168.12.10`__ incoming to __`f0/0`__

### configuration
```
!r1
conf t
    ip access-list standard task1
          10 deny host 192.168.33.100 
          15 permit any
          exit
    int e1/0
          ip access-group task1 out 
end

!r3
conf t 
    ip access-list standard task2
          deny host 192.168.12.10
          permit any 
    int f0/0
          ip access-group task2 in 
end

```

## Test
```
web> ping 192.168.11.10 -count 3
*12.0.0.1 icmp_seq=1 ttl=253 time=39.705 ms (ICMP type:3, code:13, Communication administratively prohibited)
*12.0.0.1 icmp_seq=2 ttl=253 time=36.004 ms (ICMP type:3, code:13, Communication administratively prohibited)
*12.0.0.1 icmp_seq=3 ttl=253 time=25.875 ms (ICMP type:3, code:13, Communication administratively prohibited)

web> ping 192.168.11.20 -count 3
*12.0.0.1 icmp_seq=1 ttl=253 time=30.671 ms (ICMP type:3, code:13, Communication administratively prohibited)
192.168.11.20 icmp_seq=2 timeout
*12.0.0.1 icmp_seq=3 ttl=253 time=26.802 ms (ICMP type:3, code:13, Communication administratively prohibited)

PC3> ping 192.168.33.200 -c 3
*23.0.0.3 icmp_seq=1 ttl=253 time=41.036 ms (ICMP type:3, code:13, Communication administratively prohibited)
*23.0.0.3 icmp_seq=2 ttl=253 time=26.191 ms (ICMP type:3, code:13, Communication administratively prohibited)
*23.0.0.3 icmp_seq=3 ttl=253 time=26.414 ms (ICMP type:3, code:13, Communication administratively prohibited)

PC4> ping 192.168.33.100 -c 3
84 bytes from 192.168.33.100 icmp_seq=1 ttl=61 time=38.537 ms
84 bytes from 192.168.33.100 icmp_seq=2 ttl=61 time=35.725 ms
84 bytes from 192.168.33.100 icmp_seq=3 ttl=61 time=35.879 ms

PC4> ping 192.168.33.200 -c 3
84 bytes from 192.168.33.200 icmp_seq=1 ttl=61 time=40.162 ms
84 bytes from 192.168.33.200 icmp_seq=2 ttl=61 time=36.228 ms
84 bytes from 192.168.33.200 icmp_seq=3 ttl=61 time=35.443 ms



```